#### 개요
학습용 또는 온라인 시험용을 생성한 샘플 데이터를 전처리에 사용할 수 있도록 레코드와 컬럼을 추출하고, 공사비 단위로 설비 갯 수를 계산하는 역할을 수행
* 온라인/배치 부분이 구분되어 구현되 있음

##### 진행사항
* STEP01: 각 데이터 셋에서 학습대상 레코드와 컬럼 추출
  * 이 부분은 온라인과 배치 부분이 상반되기 때문에 별도로 구현함
* STEP02: 공사비별 전주/전선/인입선 갯 수 계산 및 체크

In [1]:
import pandas as pd
from io import StringIO

import aidd.sys.config as cfg
from aidd.sys.utils import Logs
from aidd.batch.data_manager import get_provide_data, save_data

In [2]:
pdata = get_provide_data()

[9d3e88969791][2024-03-14 11:12:29.799675] 제공받은 데이터 불러오기 시작
[9d3e88969791][2024-03-14 11:12:56.867093]   읽은 데이터: CONS, 데이터 크기: (19052, 143), 처리시간: 0:00:27.067308
[9d3e88969791][2024-03-14 11:13:20.501055]   읽은 데이터: POLE, 데이터 크기: (38533, 63), 처리시간: 0:00:23.633765
[9d3e88969791][2024-03-14 11:13:50.770525]   읽은 데이터: LINE, 데이터 크기: (40019, 77), 처리시간: 0:00:30.269299
[9d3e88969791][2024-03-14 11:14:03.311539]   읽은 데이터: SL, 데이터 크기: (22632, 57), 처리시간: 0:00:12.540767
[9d3e88969791][2024-03-14 11:14:03.311644] 제공받은 데이터 불러오기 종료, 최종 처리시간: 0:01:33.511968


#### STEP01: 각 데이터 셋에서 학습대상 레코드와 컬럼 추출

##### 배치 작업

In [3]:
# 모델에 사용할 데이터 딕셔너리
batch_dict = {}

# 공사비 데이터 학습대상 레코드/컬럼 추출
key = 'CONS'
df = pdata[key]
# (전주/전선 수를 제외한) 공사비 데이터 부분에서 학습 대상 레코드 조건
# * 접수종류명(ACC_TYPE_NAME), 계약전력(CONT_CAP),
# * 공사형태코드(CONS_TYPE_CD), 총공사비(TOTAL_CONS_COST)
modeling_records = \
    (df.ACC_TYPE_NAME == cfg.COND_ACC_TYPE_NAME) & \
    (df.CONT_CAP < cfg.COND_MAX_CONT_CAP) & \
    (df.CONS_TYPE_CD == cfg.COND_CONS_TYPE_CD) & \
    (df.TOTAL_CONS_COST < cfg.COND_MAX_TOTAL_CONS_COST)
df = df[modeling_records].reset_index(drop=True)
cons_df = df[cfg.MODELING_COLS[key]]
batch_dict[key] = cons_df

# 전주/전선/인입선 데이터 학습대상 레코드/컬럼 추출
# * 레크도는 공사비에 있는 공사번호를 대상으로 함
for key in cfg.DATA_TYPE[1:]:
    df = pdata[key]
    df = df[df.CONS_ID.isin(cons_df.CONS_ID)]
    batch_dict[key] = df[cfg.MODELING_COLS[key]]
    
# (참조용)데이터 셋 별 공사대상 레코드/컬럼 크기 확인
for key in cfg.DATA_TYPE:
    print(f'{key}: {batch_dict[key].shape}')

CONS: (15335, 8)
POLE: (27896, 6)
LINE: (30348, 11)
SL: (17034, 6)


##### 온라인 작업

In [4]:
def get_online_sample(data):
    """온라인 서비스 샘플 데이터 생성 함수
    * 온라인 서비스나 온라인 서비스를 위한 전처리 프로세스 테스트를 위한
      샘플 데이터를 생성하는 함수로, 대상 데이터는 `config`에 지정되어 있음
    * 실제 온라인 작업에선 사용하지 않음
    Args:
        data (dict): 학습용으로 제공받은 전체 데이터
            - `key: df`로 구성된 딕셔너리로 제공됨
    Returns:
        jsons (dict): Online으로 받을 예정인 JSON 데이터의 딕셔너리
    """
    # 전체 데이터에서 샘플 데이터만 추출한 데이터프레임 딕셔너리
    jsons = {}
    for key in cfg.DATA_TYPE:
        df = data[key]
        df = df[df.CONS_ID.isin(cfg.CHECK_CONS_IDS)]
        # 공사비 데이터셋에 있는 총공사비를 제거하는 코드(3줄이나 소요됨;;)
        modeling_cols = cfg.MODELING_COLS[key]
        if 'TOTAL_CONS_COST' in modeling_cols:
            modeling_cols.remove('TOTAL_CONS_COST')
        jsons[key] = df[modeling_cols].to_json(orient='records')
        
    return jsons

In [5]:
json_dict = get_online_sample(data=pdata)

# 이 부분은 추후에 수정할 사항이 많아 보임
# RESTful로 입력받은 json데이터를 데이터프레임으로 변환

# 입력된 데이터를 데이터프레임으로 변환한 딕셔너리
# 모델에 사용할 데이터 딕셔너리
online_dict = {}
for key in json_dict.keys():
    online_dict[key] = pd.read_json(StringIO(json_dict[key]))
# 총공사비가 없는 경우에는 총공사비 컬럼을 0으로 추가
# 예측해야할 값이긴 하지만 온라인/배치 전처리를 같은 함수로 하기 위해 
# 컬럼만 추가함
if 'TOTAL_CONS_COST' not in online_dict['CONS'].keys():
    online_dict['CONS']['TOTAL_CONS_COST'] = 0
    
# (참조용)데이터 셋 별 공사대상 레코드/컬럼 크기 확인
for key in cfg.DATA_TYPE:
    print(f'{key}: {online_dict[key].shape}')

CONS: (3, 8)
POLE: (9, 6)
LINE: (9, 11)
SL: (3, 6)


#### STEP02: 공사비별 설비 갯 수 계산 및 체크

##### 공통 클래스 작성

In [6]:
class MergingSourceData():
    def __init__(self, pdata=None, is_batch=True):
        self._logs = Logs('BATCH_MERGE')
        self.pdata = pdata
        self.is_batch = is_batch
        self._run()
    
    def _run(self):
        self._compute_and_check_facilities_count()
        self._save_data()
        self._logs.stop()
        
    def _compute_and_check_facilities_count(self):
        """ 공사번호별 전주/전선/인입선 갯 수 계산 및 10개 이하만 추출
        """
        mdf = self.pdata[cfg.DATA_TYPE[0]]
        # 공사번호별 전주 갯 수 계산
        for key in cfg.DATA_TYPE[1:]:
            df = self.pdata[key]
            count_cons_ids = df.CONS_ID.value_counts()
            col_name = f'{key}_CNT'
            mdf = pd.merge(
                mdf, count_cons_ids.rename(col_name),
                left_on='CONS_ID', right_on=count_cons_ids.index, how='left'
            )
            # 해당 공사번호에 없는 설비는 NaN처리되며, 이 값을 0으로 변경
            # 이 부분은 전처리 단계에서 일괄 처리하니 생략함
            # mdf[col_name].fillna(0, inplace=True)  
        self._logs.mid('BEFORE_DEL_SIZE', mdf.shape)
            
        # 모델 학습에 사용할 레코드 추출
        # * 전주/전선 갯 수가 10개 이상인 경우 
        modeling_records = \
            (mdf.POLE_CNT >= cfg.COND_MIN_POLE_COUNT) & \
            (mdf.POLE_CNT <= cfg.COND_MAX_POLE_COUNT) & \
            (mdf.LINE_CNT >= cfg.COND_MIN_LINE_COUNT) & \
            (mdf.LINE_CNT <= cfg.COND_MAX_LINE_COUNT)
        mdf = mdf[modeling_records].reset_index(drop=True)
        self._logs.mid('AFTER_DEL_SIZE', mdf.shape)
        
        # 제공받은 공사비정보에 갯 수 정보를 추가함
        self.pdata['CONS'] = mdf
        
    def _save_data(self):
        # IPYNB 작업 시에만 필요(PY에서는 굳이 할 필요 없음)
        pmode = 'MB' if self.is_batch else 'MO'
        for key in cfg.DATA_TYPE:
            save_data(self.pdata[key], pmode=pmode, file_code=key)

##### 배치/온라인 작업

In [7]:
batch_md = MergingSourceData(batch_dict)
online_md = MergingSourceData(online_dict, is_batch=False)

[e9127470d8cf][2024-03-14 11:14:03.541076] 전처리를 위해 제공받은 데이터 병합 시작
[e9127470d8cf][2024-03-14 11:14:03.594582]   공사번호별 데이터 병합 결과: (15335, 11)
[e9127470d8cf][2024-03-14 11:14:03.597794]   전주/전선 10개 이상 데이터 삭제 후 결과: (14729, 11)


[e9127470d8cf][2024-03-14 11:14:04.008222] 전처리를 위해 제공받은 데이터 병합 종료, 최종 처리시간: 0:00:00.467144
[d80adaecbf22][2024-03-14 11:14:04.008462] 전처리를 위해 제공받은 데이터 병합 시작
[d80adaecbf22][2024-03-14 11:14:04.013953]   공사번호별 데이터 병합 결과: (3, 11)
[d80adaecbf22][2024-03-14 11:14:04.015191]   전주/전선 10개 이상 데이터 삭제 후 결과: (3, 11)
[d80adaecbf22][2024-03-14 11:14:04.017794] 전처리를 위해 제공받은 데이터 병합 종료, 최종 처리시간: 0:00:00.009329
